In [1]:
import gym 
from typing import Tuple
import numpy as np
import time, math, random
from sklearn.preprocessing import KBinsDiscretizer

env = gym.make('CartPole-v1')

In [2]:
policy = lambda obs: 1 

for _ in range(5):
    obs = env.reset()
    for _ in range(80):
        actions = policy(obs)
        obs, reward, done, info = env.step(actions) 
        env.render()
        time.sleep(0.05)

env.close()

2022-10-18 11:20:34.854 Python[13065:563020] ApplePersistenceIgnoreState: Existing state will not be touched. New state will be written to /var/folders/3c/5wd0mzw124926p_h14kzljq00000gn/T/org.python.python.savedState
/usr/local/lib/python3.10/site-packages/gym/logger.py:34: UserWarning: WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


In [3]:
?env.env

Type:        CartPoleEnv
String form: <CartPoleEnv<CartPole-v1>>
File:        /usr/local/lib/python3.10/site-packages/gym/envs/classic_control/cartpole.py
Docstring:  
Description:
    A pole is attached by an un-actuated joint to a cart, which moves along
    a frictionless track. The pendulum starts upright, and the goal is to
    prevent it from falling over by increasing and reducing the cart's
    velocity.

Source:
    This environment corresponds to the version of the cart-pole problem
    described by Barto, Sutton, and Anderson

Observation:
    Type: Box(4)
    Num     Observation               Min                     Max
    0       Cart Position             -4.8                    4.8
    1       Cart Velocity             -Inf                    Inf
    2       Pole Angle                -0.418 rad (-24 deg)    0.418 rad (24 deg)
    3       Pole Angular Velocity     -Inf                    Inf

Actions:
    Type: Discrete(2)
    Num   Action
    0     Push cart to the left


In [4]:
policy = lambda _,__,___, tip_velocity : int( tip_velocity > 0 )

In [5]:
n_bins = ( 6 , 12 )
lower_bounds = [ env.observation_space.low[2], -math.radians(50) ]
upper_bounds = [ env.observation_space.high[2], math.radians(50) ]

def discretizer( _ , __ , angle, pole_velocity ) -> Tuple[int,...]:
    #Convert continues state intro a discrete state
    est = KBinsDiscretizer(n_bins=n_bins, encode='ordinal', strategy='uniform')
    est.fit([lower_bounds, upper_bounds ])
    return tuple(map(int,est.transform([[angle, pole_velocity]])[0]))

In [6]:
Q_table = np.zeros(n_bins + (env.action_space.n,))
Q_table.shape

(6, 12, 2)

In [7]:
def policy( state : tuple ):
    #Choosing action based on epsilon-greedy policy
    return np.argmax(Q_table[state])

In [8]:
def new_Q_value( reward : float ,  new_state : tuple , discount_factor=1 ) -> float:
    #Temperal diffrence for updating Q-value of state-action pair
    future_optimal_value = np.max(Q_table[new_state])
    learned_value = reward + discount_factor * future_optimal_value
    return learned_value

In [9]:
# Adaptive learning of Learning Rate
def learning_rate(n : int , min_rate=0.01 ) -> float  :
   #Decaying learning rate
    return max(min_rate, min(1.0, 1.0 - math.log10((n + 1) / 25)))

In [10]:
def exploration_rate(n : int, min_rate= 0.1 ) -> float :
    #Decaying exploration rate
    return max(min_rate, min(1, 1.0 - math.log10((n  + 1) / 25)))

In [11]:
n_episodes = 10000
for e in range(n_episodes):
    
    # Siscretize state into buckets
    current_state, done = discretizer(*env.reset()), False
    
    while done==False:
        
        # policy action 
        action = policy(current_state) # exploit
        
        # insert random action
        if np.random.random() < exploration_rate(e) : 
            action = env.action_space.sample() # explore 
         
        # increment enviroment
        obs, reward, done, _ = env.step(action)
        new_state = discretizer(*obs)
        
        # Update Q-Table
        lr = learning_rate(e)
        learnt_value = new_Q_value(reward , new_state )
        old_value = Q_table[current_state][action]
        Q_table[current_state][action] = (1-lr)*old_value + lr*learnt_value
        
        current_state = new_state
        
        # Render the cartpole environment
        env.render()

KeyboardInterrupt: 

: 